## Week 2: Build The Ananlysis Engine ###

Goal: Takes raw budget data and identifies which variances acutally matter for decision-making.

### The Code ###

#### Libraries ####

In [1]:
import pandas as pd
import numpy as np

#### Load Data ####

Pulls in the 2023-2024 federal budget file with estimated vs actual spenidng. 

In [2]:
data = pd.read_csv("../data/federal_budget_2023_2024.csv")
print (data.head())

   Year     Category  Estimated Budget  Actual  Variance_Amt  Variance_Pct  \
0  2023     Interest             395.5   658.3         262.8         66.45   
1  2023     Pensions            1401.5  1459.1          57.6          4.11   
2  2023  Health Care            1636.9  1736.1          99.2          6.06   
3  2023      Defense            1167.4  1191.2          23.8          2.04   
4  2023      Welfare             486.0   544.9          58.9         12.12   

   Amt_Threshold  Pct_Threshold Material_flag  \
0             25           0.03           Yes   
1             40           0.03           Yes   
2             50           0.05           Yes   
3             50           0.05            No   
4             75           0.10           Yes   

                                    Material_Reason  
0  Dollar and percentage variance exceed thresholds  
1  Dollar and percentage variance exceed thresholds  
2  Dollar and percentage variance exceed thresholds  
3                   

#### Cleaning and Preparing the Data ####

Standardizes column names to lowercase with underscores, removes rows without actual spending data, and converting all financial columns from text to numbers so calculations run correctly. 

In [3]:
data.columns = data.columns.str.strip().str.lower().str.replace(" ","_")
data = data[~data['actual'].isna()].copy()
num_cols = ["estimated_budget", "actual", "variance_amt", "variance_pct", "amt_threshold", "pct_threshold"]
data[num_cols] = data[num_cols].apply(pd.to_numeric, errors="coerce")

#### Recalculate Variance ####

Recalculting variance independently instead of trusting the provided variance columns. This catches calculation errors in the source file, and flags any rows where the provided variance doesn't match with my calculation.(That is within $0.10 tolerance).

In [4]:
data["calc_variance_amt"] = data["actual"] - data["estimated_budget"]
data["calc_variance_pct"] = data["calc_variance_amt"] / data["estimated_budget"]
data["variance_check"] = np.isclose(data["variance_amt"], data["calc_variance_amt"], atol = 0.1)

#### Applying Materiality Logic ####
Flag variacnes as "material" if they exceed either the dollar threshold OR the percentage threshold. This mimics how finance teams actualy prioritze, a $500K variance matters even if it's only 2%, and a 40% variance matters even if it's only $10K.

In [5]:
data["material_flag_calc"] = np.where(
    (data["calc_variance_amt"].abs() >= data["amt_threshold"]) |
    (data["calc_variance_pct"].abs() >= data["pct_threshold"]),
    True,
    False
)

Documents "why" each variance was flagged, useful for explaining decisions later and debugging threshold logic. 

In [6]:
data["material_reason_calc"] = np.select(
    [
        (data["calc_variance_amt"].abs() >= data["amt_threshold"]) &
        (data["calc_variance_pct"].abs() >= data["pct_threshold"]),
        (data["calc_variance_amt"].abs() >= data["amt_threshold"]),
        (data["calc_variance_pct"].abs() >= data["pct_threshold"]),    
    ],
    [
        "Dollar and percentage variance exceed thresholds",
        "Dollar variance exceeds threshold",
        "Percentage variance exceeds threshold"
    ],
    default="Below materiality thresholds"

)

#### Assigining Tier ####

Assigns each budget category a priority tier (1 = highest) based on financial impact and organizational importance. 
Tier 1 categories (Interest, Pensions, Healt Care) are non-negotiable or politically sensitive. Tier 2 and 3 have more flexibility. The check at the end ensures no categories are accidentally left unranked.

In [7]:
tier_map = {
    "Interest": 1,
    "Pensions": 1,
    "Health Care": 1,
    "Defense": 2,
    "Education": 2,
    "Welfare": 3,
    "Transportation": 3,
    "General Government": 3,
    "Protection": 3,
    "Other Spending": 3, 
}

data["tier"] = data["category"].map(tier_map)

if data["tier"].isna().any():
    print("Catergory missing tier assignment")

#### Selecting Primary Issue (4 Step Process) ####

Narrows down all material variances to the single most important issue using a decision hierarchy:
- Only look at material variances,
- Prioritize by tier (Tier 1 beats Tier 2 even if smaller),
- Within that tier, pick the largest dollar variance,
- If there's still a tie, pick the biggest percentage variance, then most recent year, then alphabetically by category.

This ensures the brief focuses on one clear decision point instead of overwhelming leadership with a laundry list. 

Step 1 -Filtering to material only

In [8]:
mat_data = data[data["material_flag_calc"]].copy()

Step 2 - Choosing highest-priority tier present

In [9]:
sel_tier = mat_data["tier"].min()
tier_data = mat_data[mat_data["tier"] == sel_tier].copy()

Step 3- Largest absolute dollar variance

In [10]:
tier_data["abs_variance"] = tier_data["calc_variance_amt"].abs()
max_var = tier_data["abs_variance"].max()
top = tier_data[tier_data["abs_variance"] == max_var]

Step 4 - Tie-Breakers

In [11]:
def break_tie(data_ties):
    data_ties = data_ties.sort_values(
        by=[
            "calc_variance_amt",
            "calc_variance_pct",
            "year",
            "category"
        ],
        ascending=[False, False, False, True]
    )
    return data_ties.iloc[0]

main_issue = break_tie(top)

main_issue

year                                                                2023
category                                                        Interest
estimated_budget                                                   395.5
actual                                                             658.3
variance_amt                                                       262.8
variance_pct                                                       66.45
amt_threshold                                                         25
pct_threshold                                                       0.03
material_flag                                                        Yes
material_reason         Dollar and percentage variance exceed thresholds
calc_variance_amt                                                  262.8
calc_variance_pct                                               0.664475
variance_check                                                      True
material_flag_calc                                 

#### Identifying Contributing Drivers ####

Calculates what percentage of the total budget deviation each category contributed in the same year as the primary issue. This answers "what else is moving?" ad shows whether the problem is isolated or systemic. 

For Example, if Interest accounts for 34% of all variance and Education adds another 30%, leadership knows this isn't a one-off issue, it's broader fiscual pressure. 

The top 5 contributors between the "Why it Changed" section of the brief. 


#### Design Note: ####
This uses absolute variance (treating overrunns and underruns equally) beacause decision makers/ leaders need to see the total budget instability, not just spending increases. 
If the organization only cares about overruns, we would filter to "calc_variance_amt > 0" first, but for federal budget analysis, blanaced variance view is more appropriate. 

In [12]:
analysis_year = main_issue["year"]

y_data = data[data["year"] == analysis_year].copy()

y_data["abs_variance"] = y_data["calc_variance_amt"].abs()

total_var = y_data["abs_variance"].sum()

if total_var > 0:
    y_data["contribution_pct"] = y_data["abs_variance"] / total_var
else:
    y_data["contribution_pct"] = 0

driver_cont = (
    y_data.sort_values("contribution_pct", ascending=False)
    .loc[:, ["category", "abs_variance", "contribution_pct", "tier"]]
    .head(5)
)

driver_cont

,category,abs_variance,contribution_pct,tier
0,Interest,262.8,0.342232,1
9,Education,229.6,0.298997,2
2,Health Care,99.2,0.129183,1
4,Welfare,58.9,0.076703,3
1,Pensions,57.6,0.075010,1


#### Tier Summary ####
The tier-level rollup aggregate variance by priority tier, showing whether mandorty programs (Tier 1) or discretionary spending (Tier 2/3) is driving instability. This lets one write executive statements like "Mandatory programs drove 54% of total variance, indicating structual budget pressure beyond management control."

In [13]:
t_summary = y_data.groupby("tier").agg({
    "abs_variance": "sum"
}).reset_index()
t_summary["tier_contribution_pct"] = t_summary["abs_variance"] / total_var
t_summary = t_summary.sort_values("tier")

t_summary

,tier,abs_variance,tier_contribution_pct
0,1,419.6,0.546425
1,2,253.4,0.329991
2,3,94.9,0.123584


#### Generating Driver Narrative ####

Automatically generates a plain-language explanation of the top contributing factor. Instead of making someone interpret "contribution_pct: 0.3422," it produces an executive - ready sentence: " Interest accounted for 34% of the total budget variance in 2023, making it the dominant driver." This becomes the opening line for the "Why it Changed" section in the decision brief. 

In [14]:
t_driver = driver_cont.iloc[0]

driver_sent = (
    f"{t_driver['category']} accounted for "
    f"{t_driver['contribution_pct']:.0%} of total budget variance in "
    f"{analysis_year}, making it the dominant driver."
)

driver_sent

'Interest accounted for 34% of total budget variance in 2023, making it the dominant driver.'

#### Supporting Context (Other big variances) ####

Identifies up to 2 additional material variances that are at least 50% as large as the primary/main issue. This prevents the brief from being tunnel-visioned on one problem when there are other significant issues that need attention. 

For example, if Interest variance is $262.8B and Education is $229.6B (87% as large), the leadership needs to know both are moving, not just the top one. This become supporting bullets in the "What Changed" section.

In [15]:
mat_data["abs_variance"] = mat_data["calc_variance_amt"].abs()

supp = mat_data[
    (mat_data.index != main_issue.name) &
    (mat_data["abs_variance"] >= 0.5 * main_issue["abs_variance"])
].sort_values("abs_variance", ascending=False).head(2)

#### Escalation Trigger Logic ####

Determines whether the budget situation requires executive section based on two thresholds: 
- 5 or more material variances across different categories signals widespread instability, or
- total variance exceeding 10% of the entire budhet indicates systemic fiscal stress.

This prevents the brief from treating a $200B Interest overrun the same way whether it's an isolated issue or part of a borader budget crisis. The flag triggers different language in the "Recommendation Actions" section, isolated issues get tactical fixes, sustemic problesm get strategic escalation.

In [16]:
mat_count = mat_data.shape[0]

t_budget = data["estimated_budget"].sum()
t_variance = data["calc_variance_amt"].abs().sum()

flag = (mat_count >= 5) or (total_var / t_budget >= 0.10)


#### Confidence Scoring Engine ####

Assesses data quality by checking how much acutal spending data is missing. Awards 30 points if the dataset is complete, 15 points if less than 10% is missing (minor gaps are tolerable), and 0 points if more than 10% is missing (too unreliable for confident decisions). This becomes part of the overall confidence score that tells leadership whether to act immediately or gather more data first. A breif based on 95 % complete data gets flagged differently than one based on 60% complete data. 

In [17]:
missing_pct = data["actual"].isna().mean()

if missing_pct == 0:
    dq_score = 30
elif missing_pct < 0.10:
    dq_score = 15
else: 
    dq_score = 0

missing_pct

0.0

#### Pattern Consistency (30) ####

Checks whether the primary/main issue's variance is a consistent pattern or a one-time anomaly by comparing it to the previous year. Awards 30 points if the variance direction matches (e.g., both year overspent), 0 points if it flips (overspent last year, underspent this year, suggests volatility) and 15 points if there's only one year of data (can't confirm pattern). 

- Consistent patterns = higher confidence in root cause.
- Random flucations = lower confidence, needs more investigation.

In [18]:
categ_hist = data[data["category"] == main_issue['category']]

if categ_hist.shape[0] >= 2:
    direction = np.sign(categ_hist.iloc[0]["calc_variance_amt"]) == \
                     np.sign(categ_hist.iloc[1]["calc_variance_amt"])
    patt_score = 30 if direction else 0
else:
    patt_score = 15

categ_hist

,year,category,estimated_budget,actual,variance_amt,variance_pct,amt_threshold,pct_threshold,material_flag,material_reason,calc_variance_amt,calc_variance_pct,variance_check,material_flag_calc,material_reason_calc,tier
0,2023,Interest,395.5,658.3,262.8,66.45,25,0.03,Yes,Dollar and percentage variance exceed thresholds,262.8,0.664475,True,True,Dollar and percentage variance exceed thresholds,1
10,2024,Interest,788.8,879.9,91.1,11.55,25,0.03,Yes,Dollar and percentage variance exceed thresholds,91.1,0.115492,True,True,Dollar and percentage variance exceed thresholds,1


#### Driver Clarity (30) ####

Measures how clear-cut the root cause is. Awards 30 points if there's only one dominant issue (easy to diagnose and fix), 15 points if there are 2-3 significant variances (still manageable but more complex), and 0 points if there are many large variances (messy situation, harder to pinpoint cause). 

A single $200B Interest overrun gets higher confidence than five different $50B variances across categories, the first has a clear driver, the second suggests something systemic or data problems. 

In [19]:
if supp.empty:
    driver_score = 30
elif supp.shape[0] <= 2:
    driver_score = 15
else:
    driver_score = 0

driver_score

15

#### Magnitude ####

Awards confidence points based on how large the variance is as a percentage. Gives 10 points if the variance is 20%+ (impossible to miss, clearly significant), 5 points if it's 10-20% (material but could have legitimate explanations), and 0 points if it's under 10% (might be within normal operational fluctuation). 

Larger variances = higher confidence that this is a real issue worth acting on, not just statistical noise or minor timing differences.

In [20]:
abs_pct = abs(main_issue["calc_variance_pct"])

if abs_pct >= 0.20:
    mag_score = 10
elif abs_pct >= 0.10:
    mag_score = 5
else: 
    mag_score = 0

abs_pct

0.6644753476611882

#### Final Confidence ####

Combines all four scoring dimensions (data quality, pattern consistency, driver clarity, magnitude) into a single 0-100 confidence score, then translates it into executive language:
- High (85+),
- Medium (60-84), or
- Low (<60).

This tells how much to trust the analysis. 
- High confidence = act it now.
- Medium confidence = probably real but verify key assumptions.
- Low confidence = gather more data before making decisions.

The confidence band appears prominently in the decision brief so leaders know whether they're working with solid evidence or preliminary signals.

In [21]:
confi_score = dq_score + patt_score + driver_score + mag_score 

if confi_score >= 85:
    confi_band = "High"
elif confi_score >= 60:
    confi_band = "Medium"
else:
    confi_band = "Low"

confidence = {
    "score": confi_score,
    "band": confi_band
}
confidence

{'score': 85, 'band': 'High'}

## Week 3 ##

In [32]:
from pathlib import Path

OUTPUT_DIR = Path("../output")
OUTPUT_DIR.mkdir(exist_ok=True)

#### Driver Summary Table ####

Creates a clean ranked table showing all catergories by their variance contribution for the issue year. This becomes an appendix or supporting table in the decision breif, a quick scan to see the full picture beyond just the top 3-5 drivers mentioned in the narrative. Sorted by absolute variance so the biggest movers appear first, with tier labels to show priority classification.

In [33]:
year = main_issue["year"]

y_data = data[data["year"] == year].copy()

y_data["abs_variance"] = y_data["calc_variance_amt"].abs()

total_var = y_data["abs_variance"].sum()

y_data["contribution_pct"] = y_data["abs_variance"] / total_var

table = (
    y_data[["category", "abs_variance", "contribution_pct", "tier"]]
    .sort_values("abs_variance", ascending=False)
    .reset_index(drop=True)
)

table

,category,abs_variance,contribution_pct,tier
0,Interest,262.8,0.342232,1
1,Education,229.6,0.298997,2
2,Health Care,99.2,0.129183,1
3,Welfare,58.9,0.076703,3
4,Pensions,57.6,0.075010,1
5,Defense,23.8,0.030994,2
6,Transportation,21.5,0.027998,3
7,Other Spending,10.4,0.013543,3
8,General Government,2.3,0.002995,3
9,Protection,1.8,0.002344,3


#### Executive Summary Generator ####

Automatically writes the opening paragraph of the decision brief by pulling together the primary/main issue, its magnitude, the top contributing driver, and the confidence level. Converts raw numbers into executive language,i.ie., "exceeded the budget by $262.8B (66.4%)" instead of "variance_amt: 262.8". This 3-4 snetence summary highlights for deciding whether to keep reading or take immediate action. 

In [34]:
def fmt_b(x):
    return f"${abs(x):,.1f}B"

def gen_summary (main, table, confidence):
    status = "exceeded" if main ["calc_variance_amt"] > 0 else "came in under"

    pct = abs(main["calc_variance_pct"]) * 100
    amt = abs(main["calc_variance_amt"])

    t_driver = table.iloc[0]

    summary = (
        f"{main['category']} spending {status} the budget by "
        f"{fmt_b (amt)} ({pct:.1f}%) in FY{int(main['year'])}. " 
        f"This was the largest material variance among all categories. "
        f"{t_driver['category']} accounted for "
        f"{t_driver['contribution_pct']*100:.0f}% of total budget deviation. "
        f"Confidence in this assessment is {confidence['band'].lower()}. "
    )
    return summary

print(gen_summary (main_issue, table, confidence))

Interest spending exceeded the budget by $262.8B (66.4%) in FY2023. This was the largest material variance among all categories. Interest accounted for 34% of total budget deviation. Confidence in this assessment is high. 


#### "What Changed" Section Generator ####

Generates the factual "what happened" section by stating the budgeted amount, acutal amount, variance, and priority tier. Keeps its neutral and data- driven with no interpretation yet, just the numbers in plain language. This gives the baseline facts before diving into drivers and recommendations.

In [35]:
def changed(main):
    return(
        f"Budgeted spending for {main['category']} was {fmt_b(main['estimated_budget'])}, "
        f"while actual spending reached {fmt_b(main['actual'])}. "
        f"The resulting variance was {fmt_b(main['calc_variance_amt'])}, "
        f"or {abs(main['calc_variance_pct'])*100:.1f}% versus plan. " 
        f"This category is Tier {main['tier']} in fiscal priority. "
     )

#### "Why It Changed" Section Generator ####

Explains the drivers behind the variance by identifying the top contributing categories and their relative impact. 

If one category dominates (60%+ of total variance), it states that clearly that is the focused issue. 

If there are multiple significant contributions with more balanced percentages, it flags "broader budget execution pressure" to signal systemic instability rather than an isolated problem. 

This helps in understanding what kind of issue is been dealed, single fixable issue or a more complex situation requiring broader intervention. 

In [36]:
def why_changed(table, main_issue):
    top = table.iloc[0]
    second = table.iloc[1] if len(table) > 1 else None

    t_cat = top["category"]
    t_pct = round(top["contribution_pct"] * 100)

    if second is None:
        return (
            f"The variance was driven almost entirely by {t_cat}, "
            f"which accounted for {t_pct}% of total deviation. "
        )

    second_cat = second["category"]
    second_pct = round(second["contribution_pct"] * 100)

    if t_pct >= 60:
        return (
            f"The variance was primarily driven by {t_cat} ({t_pct}%), "
            f" with limited contribution from other categories. "
        )

    return (
        f"Variance was driven by multiple categories, led by {t_cat} ({t_pct}%) "
        f"and {second_cat} ({second_pct}%), indicating broader budget execution pressure. "
    )

#### Recommended Actions Generator ####

Generates tailored recommendations based on the issue type and category priority.

For Tier 1 overruns (Interest, Pensions, Health Care, > 20%), suggests strategic actions like reviewing structual drivers and updating multi-year forecasts, there aren't discretionary cuts, they required policy-level responses. 

For Tier 2/3 overrunns, it recommends operational efficiency and execution controls. For smaller variances, suggests routine monitoring without immediate action. 

This ensures recommendations match the acutal severity, as 5% deviation can't be treated as the same as 66% overrun, and can't recommend "cut spending" for mandatory programs where that's not an option. 

In [37]:
def gen_action(main):
    tier = main["tier"]
    pct = abs(main["calc_variance_pct"]) * 100
    direction = "over" if main["calc_variance_amt"] > 0 else "under"
    category = main["category"]

    if tier == 1 and direction == "over" and pct > 20:
        return (
            f"Initiate immediate cost containment review for {category}. "
            "Research mandatory or policy-driven cost drivers and updared forward budget assumptions."
            "Escalate structual funding risks for executive or legislative review."
        )

    if tier == 1 and direction == "over":
        return (
            f"Strengthen in-year monitroing for {category} and refine forecasting assumptions. "
            "Evaluate policy sensitivity and interest-rate or demand exposure."
        )
    if tier == 2:
        return (
            f"Target operational efficiency improvements withtin {category}. "
            "Apply tighter execution controls and monitor variance trajectory."
        )
    
    return (
        f"Continue routine monitoring of {category}. "
        "No immeidate corrective action required at current variance levels."
    )


#### Confidence Disclosure Generator ####

Transparently discloses the confidence level and score so it can be known how much weight to give the recommendations. 

A "High(90%)" brief gets acted on immediately, while a "Medium(65%)" breif might trigger additional verification first. This prevents overconfident decision-making when the underlying analysis has gaps or uncertainty.

In [38]:
def gen_confi(confidence):
    return (
        f"Overall confidence is {confidence['band']} ({confidence['score']}%). "
        "Data quality is sufficient and variance drivers are reasonably well understood."
    )

#### Full Decision Brief Assembly ####

Assembles all the individual narrative sections into the final one-page decision brief. Pulls together the executive summary, factual variance details, driver analysis, tailored recommendations, and confidence disclosure into a structured format that matched with acutal consume information. 

In [39]:
def gen_decision(main, table, confidence):
    return f"""
DECISION BRIEF - FY{int(main['year'])}

EXECUTIVE SUMMARY
{gen_summary(main, table, confidence)}

WHAT CHANGED
{changed(main)}

WHY IT CHANGED
{why_changed(table, main)}

RECOMMENDED ACTIONS
{gen_action(main)}

CONFIDENCE AND ASSUMPTIONS
{gen_confi(confidence)}
"""

print(gen_decision(main_issue, table, confidence))


DECISION BRIEF - FY2023

EXECUTIVE SUMMARY
Interest spending exceeded the budget by $262.8B (66.4%) in FY2023. This was the largest material variance among all categories. Interest accounted for 34% of total budget deviation. Confidence in this assessment is high. 

WHAT CHANGED
Budgeted spending for Interest was $395.5B, while actual spending reached $658.3B. The resulting variance was $262.8B, or 66.4% versus plan. This category is Tier 1 in fiscal priority. 

WHY IT CHANGED
Variance was driven by multiple categories, led by Interest (34%) and Education (30%), indicating broader budget execution pressure. 

RECOMMENDED ACTIONS
Initiate immediate cost containment review for Interest. Research mandatory or policy-driven cost drivers and updared forward budget assumptions.Escalate structual funding risks for executive or legislative review.

CONFIDENCE AND ASSUMPTIONS
Overall confidence is High (85%). Data quality is sufficient and variance drivers are reasonably well understood.



#### HTML and PDF Export ####

In [40]:
!pip install playwright
!python -m playwright install chromium


[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


#### Brief Generation Workflow ####

Runs the complete analysis pipeline for each Fiscal Year separately. For each year, it :
- Filters to that year's data.
- Identifies material variances.
- Selects the primary issue using tier prioritization and variance magnitude.
- Calculates how much each category contributed to total variance.
- Scores confidence based on driver clarity and variance size.
- Packages everything into a confidence rating.

This happens independently for 2023 and 2024, so each year gets its own tailored breif based on that year's specific situation. 

#### HTML ####
Creates a formatted HTML and PDF document with clean styling and populates it with all five narrative sections. Saves it as an HTML file (e.g., Decision_Brief_FY2023.html) and PDF file (e.g., Decision_Brief_FY2023.pdf). 

The entire process runs automatically: raw data in -> decision brief out -> with zero manual formatting required.

In [41]:
import os
import asyncio
from playwright.async_api import async_playwright

In [46]:
years = [2023, 2024]

async def gen_brief():
    for year in years:
        print(f"Generating brief for FY{year}...")

        FY = data[data["year"] == year].copy()
        FY["abs_variance"] = FY["calc_variance_amt"].abs()
        
        mat_data = FY[FY["material_flag_calc"] == True]
        
        t_tier = mat_data["tier"].min()
        t_data = mat_data[mat_data["tier"] == t_tier]
        
        max_var = t_data["abs_variance"].max()
        t = t_data[t_data["abs_variance"] == max_var]
        
        main_issue = break_tie(t)
        
        table = (
            FY
            .groupby(["category", "tier"], as_index=False)["abs_variance"]
            .sum()
        )
        
        total_var = table["abs_variance"].sum()
        table["contribution_pct"] = table["abs_variance"] / total_var
        table = table.sort_values("abs_variance", ascending=False)
        
        score = 30
        patt_score = 30

        if table.shape[0] <=2:
            score = 30
        elif table.shape[0] <=4:
            score = 15
        else: 
            score = 0

        pct = abs(main_issue["calc_variance_pct"]) * 100

        if pct > 20:
            mag_score = 10
        elif pct >= 10:
            mag_score = 5
        else: 
            mag_score = 0
        confi_score = dq_score + patt_score + score + mag_score
        if confi_score >= 85:
            confidence_band = "High"
        elif confi_score >= 60:
            confidence_band = "Medium"
        else: 
            confidence_band = "Low"

        confidence = {
        "score" : confi_score,
        "band" : confi_band
        }

        html_doc = f"""
<html>
<head>
<meta charset="utf-8">
<style>
    body {{ font-family: Arial, sans-serif; margin: 40px; font-size: 11pt; line-height: 1.4; }}
    h1 {{ font-size: 18pt; margin-bottom: 10px; }}
    h2 {{ font-size: 12pt; margin-top: 16px; border-bottom: 1px solid #ddd; }}
    p {{ margin: 4px 0 10px 0;}}
</style>
</head>

<body>

<h1>DECISION BRIEF - FY{year}</h1>

<h2>EXECUTIVE SUMMARY</h2>
<p>{gen_summary(main_issue, table, confidence)}</p>

<h2>WHAT CHANGED</h2>
<p>{changed(main_issue)}</p>

<h2>WHY IT CHANGED</h2>
<p>{why_changed(table, main_issue)}</p>

<h2>RECOMMENDED ACTIONS</h2>
<p>{gen_action(main_issue)}</p>

<h2>CONFIDENCE AND ASSUMPTIONS</h2>
<p>{gen_confi(confidence)}</p>

</body>
</html>
"""

        html_f = OUTPUT_DIR / f"Decision_Brief_FY{year}.html"
        with open(html_f, "w", encoding="utf-8") as f:
            f.write(html_doc)

        print(f"Saved {html_f}")

        pdf_f = OUTPUT_DIR / f"Decision_Brief_FY{year}.pdf"
        html_path = html_f.resolve().as_uri()

        async with async_playwright() as p:
            browser = await p.chromium.launch()
            page = await browser.new_page()
            await page.goto(html_path)
            await page.pdf(
                path=pdf_f,
                format="A4",
                margin={"top": "40px", "bottom": "40px", "left": "40px", "right": "40px"},
                print_background=True
            )
            await browser.close()

        print(f"Saved {pdf_f}")

await gen_brief()

Generating brief for FY2023...
Saved ../output/Decision_Brief_FY2023.html
Saved ../output/Decision_Brief_FY2023.pdf
Generating brief for FY2024...
Saved ../output/Decision_Brief_FY2024.html
Saved ../output/Decision_Brief_FY2024.pdf
